In [ ]:
# Packages used: NetworkX -- for tree representation as directed acyclic graphs

import networkx as nx

class Compute_measures(object):
    def __init__(self, tree):
        self.tree=tree                          # tree encodes the nodes and edges content in dictionary format. It uses directed graph (DiGraph) feature of networkX package. For example, nodes are encoded like this - tree.nodes={1:{form:'this',POS:'PRN'},2:{...}}   
        self.root=0                             # ROOT is an abstract node in the tree and is encoded as empty and with name=0    
        
    def dependency_direction(self, edge):       # Computes the direction of an edge (i.e., dependency) according to relative position of dependent and head  
        direction=''
        if edge[0]>edge[1]:                     # edge is a list data type in the format [head,dependent]
            direction='RL'
        else:
            direction='LR'

        return direction                        # return direction as 'LR' (Left-to-Right) or RL ('Right-to-Left')  

    def dependency_distance(self, edge):        # Computes the dependency length i.e., no. of nodes between head and its dependent 
        dd=0
        if edge[0]>edge[1]:                      
            for nodex in nx.descendants(self.tree, self.root):        
                if edge[1]<nodex<edge[0]:                             # all the nodes that lies linearly between dependent and head   
                    dd+=1
        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    dd+=1
        return dd                              # returns the dependency distance of the edge
        
    def is_projective(self, edge):             # Checks if an edge is projective or not and returns a boolean value.
        projective=True
        edge_span=[]                 
        if edge[0]>edge[1]:                                      
            for nodex in nx.descendants(self.tree, self.root):   
                if edge[1]<nodex<edge[0]:
                    edge_span.append(nodex)                       
        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    edge_span.append(nodex)

        flag=0
        for nodeI in edge_span:
            if not self.tree.nodes[nodeI]['head'] in edge_span:         # if the head of any intervening node exists outside the span of the edge
                if not self.tree.nodes[nodeI]['head']==edge[0]:
                    if not self.tree.nodes[nodeI]['head']==edge[1]:
                        if not self.tree.nodes[nodeI]['deprel']=='punct':
                            flag += 1
        if not flag==0:
            projective=False
        return projective
    
    def edge_degree(self, edge):                                 # Computes the number of edges causing non-projectivity              
        eD=0
        edge_span=[]                 
        if edge[0]>edge[1]:                                      
            for nodex in nx.descendants(self.tree, self.root):   
                if edge[1]<nodex<edge[0]:
                    edge_span.append(nodex)                       
        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    edge_span.append(nodex)

        for nodeI in edge_span:
            if not self.tree.nodes[nodeI]['head'] in edge_span:         # if the head of any intervening node exists outside the span of the edge
                if not self.tree.nodes[nodeI]['head']==edge[0]:
                    if not self.tree.nodes[nodeI]['head']==edge[1]:
                        if not self.tree.nodes[nodeI]['deprel']=='punct':
                            eD += 1    
        return eD                                                        
 

    def compute_all(self):
        # Arity=self.arity()
        # Projection_degree=self.projection_degree()
        # Gap_degree=self.gap_degree()
        direction={}
        dep_distance={}
        projectivity={}
        Edge_degree={}
        # endpoint_cross={}
        for edgex in self.tree.edges:
            direction[edgex]=self.dependency_direction(edgex)
            dep_distance[edgex]=self.dependency_distance(edgex)
            projectivity[edgex]=self.is_projective(edgex)
            Edge_degree[edgex]=self.edge_degree(edgex)
            # endpoint_cross[edgex]=self.endpoint_crossing(edgex)

        return [direction, dep_distance, projectivity, Edge_degree]
        # return [Arity, Projection_degree, Gap_degree, direction, dep_distance, projectivity, Edge_degree, endpoint_cross]

   

In [ ]:
# Using NetworkX package and conllu package
import os
from io import open
from conllu import parse
import networkx as nx
from operator import itemgetter
import random

directory = "./UD"                   # directory containing the UD scheme tree files in CONLLU format
ud_files = []
for root, dirs, files in os.walk(directory):     
    for file in files:
        if file.endswith('train.conllu') or file.endswith('test.conllu'):
            fullpath = os.path.join(root, file)
            ud_files.append(fullpath)            # creates a list of path of all files (file of each language) from the directory

num_sent=0
results = open('results_UD.csv','a')
results.write("#"+"\t"+"TREEBANK"+"\t"+"ID"+"\t"+"S-LENGTH"+"\t"+"EDGE"+"\t"+"DIR"+"\t"+"D-LENGTH"+"\t"+"PROJ"+"\t"+"#CROSS"+"\n")
results.close()

n_f = 0
n_files = len(ud_files)

for i in ud_files:                                       # reads file of each language one by one
    n_f += 1
    lang = str(i).replace("./UD", "")        
    lang=lang.replace("-ud-train.conllu", "")            # lang variable stores the language code
    lang=lang.replace("-ud-test.conllu", "") 
    data_file = open(str(i),'r',encoding='utf-8').read()
    sentences = []
    sentences = parse(data_file)                         # parses the CONLLU format    
    n_sent = len(sentences)       
    

    for sentence in sentences[0:]: # 0:
        num_sent += 1
        print(str(n_f)+"/"+str(n_files), str(num_sent)+"/"+str(n_sent))
        s_id = sentence.metadata['sent_id']

        tree = nx.DiGraph()                              # An empty directed graph (i.e., edges are uni-directional)  
        for nodeinfo in sentence[0:]:                    # retrieves information of each node from dependency tree in UD format     
            entry=list(nodeinfo.items())
            if not entry[7][1]=='punct':
                tree.add_node(entry[0][1], form=entry[1][1], lemma=entry[2][1], upostag=entry[3][1], xpostag=entry[4][1], feats=entry[5][1], head=entry[6][1], deprel=entry[7][1], deps=entry[8][1], misc=entry[9][1])                #adds node to the directed graph 
        ROOT=0
        tree.add_node(ROOT)                            # adds an abstract root node to the directed graph
            
        for nodex in tree.nodes:
            if not nodex==0:
                if tree.has_node(tree.nodes[nodex]['head']):                                         # to handle disjoint trees
                    tree.add_edge(tree.nodes[nodex]['head'],nodex,drel=tree.nodes[nodex]['deprel'])       # adds edges as relation between nodes
                if tree.nodes[nodex]['head']==0:
                    real_root=nodex
        
        n = max([x for x in tree.nodes if isinstance(x, int)])
        m = len(tree.edges)
        get = Compute_measures(tree)

        for edgey in tree.edges:
            if not edgey[0]==0:
                direction_real = get.dependency_direction(edgey)    # direction of the edge in terms of relative linear order of head and its dependent
                dep_distance_real=get.dependency_distance(edgey)    # gives the distance between nodes connected by an edge 
                if get.is_projective(edgey):                   # checks if edge is projective or not
                    projectivity_real=1
                else:
                    projectivity_real=0
                edge_degree_real=get.edge_degree(edgey)             # gives the no. of edges crossing an edge
                results = open('results_UD.csv','a')
                results.write(str(num_sent)+"\t"+str(lang)+"\t"+str(s_id)+"\t"+str(n)+"\t"+str(edgey)+"\t"+str(direction_real)+"\t"+str(dep_distance_real+1)+"\t"+str(projectivity_real)+"\t"+str(edge_degree_real)+"\n")
                results.close()
        # print("\n-----------------\n"+str(tree.edges))

In [ ]:
import csv
import numpy as np
from collections import defaultdict
from ast import literal_eval
import random
from tqdm import tqdm


if __name__ == "__main__":

    no_sent = 0
    sent_cnt = defaultdict(int)
    no_edges = 0
    prev_id = ''

    edge_dic = defaultdict(lambda: defaultdict(int))

    # Read data and save edge frequencies in dictionaries
    with open('./results_UD.csv', newline='') as f:
        reader = csv.reader(f, delimiter='\t')
        next(reader)        #Discard the header
        for row in reader:
            no_edges += 1
            curr_id = row[2]
            if prev_id != curr_id:      #We have a new sentence
                no_sent += 1
                prev_id = curr_id
                sent_len = int(row[3])
                sent_cnt[sent_len] += 1

            edge = tuple(map(lambda x: x-1 ,sorted(literal_eval(row[4]))))      #Reduce indices for later
            edge_dic[sent_len][edge] += 1

    arraydic = {}
    for sl in edge_dic.keys():
        arraydic[sl] = {}
        arraydic[sl]['edges'] = np.fromiter(edge_dic[sl].keys(), dtype=np.dtype('int,int'))
        arraydic[sl]['weights'] = np.fromiter(edge_dic[sl].values(), dtype=int)

        # Create mask. Note: entry 0 if the sentence-index occurs in the edge-index, and 1 if it does not
        arraydic[sl]['mask'] = mask = np.ones((sl, arraydic[sl]['edges'].size), dtype=bool)
        for i, ed in enumerate(arraydic[sl]['edges']):
            start, end = ed
            mask[start][i] = 0
            mask[end][i] = 0

    max_s = 499
    min_tokens = 1_000_000_000
    no_tokens = 0
    with open('./depstrings_UD.txt', 'x') as res, tqdm(total=min_tokens) as bar:
        while no_tokens < min_tokens:
            r_slen = random.choices(list(sent_cnt.keys()), weights=list(sent_cnt.values()), k=1)[0]
            no_tokens += r_slen
            bar.update(r_slen)
            synt_sent = np.full((r_slen), -1, dtype=int)       #Output sentence

            eds = arraydic[r_slen]['edges']
            no_eds = eds.size
            wts = arraydic[r_slen]['weights']
            mskmat = arraydic[r_slen]['mask']

            # Fill indices according to edges
            msk = np.ones((no_eds), dtype=bool)
            while len(msk_eds := eds[msk]):
                redge = random.choices(msk_eds, weights=wts[msk], k=1)[0]
                ed_s, ed_e = redge
                synt_sent[[ed_s, ed_e]] = random.randint(0, max_s+1)
                msk &= mskmat[ed_s] & mskmat[ed_e]

            # Fill remaining indices and write to file
            for symb in synt_sent:
                if symb == -1:
                    symb = random.randint(0, max_s+1)
                res.write(f"{symb} ")
            res.write("\n")

            # if not index%10_000:
            #     print(f"{no_tokens}/{min_tokens}")